In [36]:
import os

In [37]:
%pwd

'd:\\Machine learning projects\\kidney_disease_classifier_tut'

In [38]:
os.chdir(r'd:\Machine learning projects\kidney_disease_classifier_tut')
print(os.getcwd())  # This should now print the root folder of your project

d:\Machine learning projects\kidney_disease_classifier_tut


In [39]:
from dataclasses import dataclass
from pathlib import Path

#defining the kind of data that can be stored for the data ingestion config class where changing or adding anythign later would throw an error as the varaibels are frozen
@dataclass(frozen=True) 
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [40]:
from cnnClassifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [41]:
import os
print(os.getcwd())


d:\Machine learning projects\kidney_disease_classifier_tut


In [42]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,# access the config.yaml file and the params.yaml file from their paths and store them
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)# read the yaml files using the custorm read_yaml function
            self.params = read_yaml(params_filepath)# the function returns a configBox datatype 

            create_directories([self.config.artifacts_root]) # creates a directory called artifacts root as stated in the config.yaml file

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion # using the configBox accessing method we can access the data_ingestion key inside the config file

        create_directories([config.root_dir]) # create a the root directory as mentioned in the config file

        data_ingestion_config = DataIngestionConfig( # use the previously created frozen entity for managing the data ingestion variables
            root_dir = config.root_dir,# access the information stored int he config  file and read it into respective  variables one by one
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config # return the data ingestion information as defined by the dataingestionconfig class created earlier

In [43]:
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [44]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_URL = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f' Downloading data from {dataset_URL} into {zip_download_dir}')

            file_id = dataset_URL.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f'Data from {dataset_URL} downloaded successfully into {zip_download_dir}')
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [45]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-11 18:28:02,070 : INFO : common : yaml file : config\config.yaml loaded successfully]
[2025-03-11 18:28:02,071 : INFO : common : yaml file : params.yaml loaded successfully]
[2025-03-11 18:28:02,072 : INFO : common : created directory at: artifacts]
[2025-03-11 18:28:02,073 : INFO : common : created directory at: artifacts/data_ingestion]
[2025-03-11 18:28:02,074 : INFO : 3676027333 :  Downloading data from https://drive.google.com/file/d/15QjiJBAVfpVMwEqW2dBw3kqfCbPF5BLu/view?usp=sharing into artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=15QjiJBAVfpVMwEqW2dBw3kqfCbPF5BLu
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=15QjiJBAVfpVMwEqW2dBw3kqfCbPF5BLu&confirm=t&uuid=bdac9d5c-fdc2-4a82-8a63-763251072999
To: d:\Machine learning projects\kidney_disease_classifier_tut\artifacts\data_ingestion\data.zip
100%|██████████| 61.0M/61.0M [00:05<00:00, 12.1MB/s]


[2025-03-11 18:28:11,966 : INFO : 3676027333 : Data from https://drive.google.com/file/d/15QjiJBAVfpVMwEqW2dBw3kqfCbPF5BLu/view?usp=sharing downloaded successfully into artifacts/data_ingestion/data.zip]
